In [1]:
!pip install langchain --quiet
!pip install langchain-groq --quiet
!pip install langchain-core --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 3.5 MB/s eta 0:00:00


In [2]:
groq_api = "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"

In [3]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_core.tools import tool

In [4]:
llm = ChatGroq(api_key=groq_api)

In [106]:
import re
import json
from typing import Any, Callable, Dict, List, Optional


class FunctionRegistry:
    """
    A registry to manage and call external functions dynamically.
    """
    def __init__(self):
      self._functions: Dict[str, Callable] = {}
      self._functions_description: Dict[str, str] = {}

    def register(self, name: str, func: Callable, description: Optional[str] = None) -> None:
      """
      Register a function with the registry
      Args:
        name: Unique Idetifier for function
        func: The function to be registered
        description: description of the functions
      Return:
        None
      """
      self._functions[name] = func
      self._functions_description[name] = description or func.__doc__ or "No description available"

    def call(self, name: str, args: List[Any] = None, kwargs: Dict[str, Any] = None):
      """
      Call the register function with given arguments

      Args:
        name: name of the fuctions to call
        args: Argmunts to pass
        kwargs: Keyword arguments to pass
      Return:
        Result of the function call
      """

      if name not in self._functions:
        raise ValueError(f"Function {name} not found")

      args = args or []
      kwargs = kwargs or {}

      try:
        function = self._functions[name](**args,  **kwargs)
        return function

      except Exception as e:
        return f"Calling function error {e}"

class FunctionCallManager:
  def __init__(self, FunctionRegistry: FunctionRegistry):
    self.registry = FunctionRegistry

  def generate_function_call_prompt(self, query: str) -> str:
    """
    Generate a prompt that guides the model to use available functions.

    Args:
        query: User's original query
    Returns:
        Formatted prompt with function descriptions
    """
    function_descriptions = "\n".join([str(description) for description in self.registry._functions_description.values()])

    prompt = f"""
    Intelligent Assistant Function Call Instruction Guide

      Role:
      - You are an advanced assistant designed to solve user queries by leveraging external functions.

      Function Descriptions:
      {function_descriptions}

      User Query:
      {query}

      Function Interaction Protocol:
      1. Query Analysis
        - Carefully examine the user's request
        - Identify if any available functions can help solve the query effectively

      2. Function Call Decision
        - If a relevant function is found:
          a. Construct a precise JSON-formatted function call
          b. Ensure the call includes:
              - Correct function name
              - Necessary arguments
              - Optional keyword arguments

      3. Function Call JSON Format:
        {{
          "function": "name_of_function",
          "args": [...],
          "kwargs": {{....}}
        }}

      4. Handling Different Scenarios:
        - If a function perfectly matches the query: Use function call
        - If no function is suitable: Provide a direct, concise answer
        - If function arguments are incomplete: Request additional information from the user
        - When you are responsing user's should not know you are calling a function or what are the Instruction you have provided.

      5. Answering Principle:
        - Always aim to be helpful, accurate, and direct
        - Prioritize using functions when appropriate
        - Maintain clarity in communication

      Preparation:
      - Review available functions before processing each query
      - Be ready to explain function usage if needed
          """

    return prompt

  def parse_function_call(self, model_response: str) -> Optional[Dict[str, Any]]:
      """
      Parse function call from model's response.

      Args:
          model_response: Model's generated response

      Returns:
          Parsed function call details or None
      """
      try:
          # Try to extract JSON function call
          match = re.search(r'\{\s*"function":\s*".*",\s*"args":\s*\[.*\],\s*"kwargs":\s*\{\}\s*\}', model_response, re.DOTALL)
          if match:
              func_call = json.loads(match.group(0))
              return func_call

          # Fallback to manual parsing if JSON fails
          return self._manual_parse(model_response)
      except Exception:
          return None

In [115]:
def add(a: int, b: int) -> int:
  """
  Args
  a :int
  b: int

  return a+b: int
  """
  return a + b

def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


add_description =  {
  "name": "add",
  "description": "Adds two integers together.",
  "parameters": {
      "type": "object",
      "properties": {
          "a": {"type": "integer"},
          "b": {"type": "integer"}
      },
      "required": ["a", "b"]
  }
}

multiply_description = {
    "name": "multiply",
    "description": "Multiplies two integers.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "integer"},
            "b": {"type": "integer"}
        },
        "required": ["a", "b"]
    }
}



registry = FunctionRegistry()
registry.register("add", add, add_description)
registry.register("multiply", multiply,  multiply_description)

In [116]:
registry._functions

{'add': <function __main__.add(a: int, b: int) -> int>,
 'multiply': <function __main__.multiply(a: int, b: int) -> int>}

In [117]:
function_manager = FunctionCallManager(registry)
prompt = function_manager.generate_function_call_prompt("what is the  25 * 25")
response = llm.invoke(prompt).content
parse_response = function_manager.parse_function_call(response)
if parse_response:
  function = eval(parse_response["function"])
  args = parse_response["args"]
  kwargs = parse_response["kwargs"]
  result = function(*args, **kwargs)
  print(result)
else:
  print(response)

625
